# **Introduction to Parallel Programming**

Traditionally, computers were only focused on computer programs serially. These programs were broken into series instructions that are executed on a single processor sequentially. In contrast with parallel programming, you can break down the computer programs into the sub-programs which will be executed on multiple processor in parallel.

[CUDA®](https://developer.nvidia.com/cuda-zone) is a parallel computing platform and programming model developed by NVIDIA for general computing on graphical processing units (GPUs). With CUDA, developers are able to dramatically speed up computing applications by harnessing the power of GPUs.

The [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit) from NVIDIA provides everything you need to develop GPU-accelerated applications. The CUDA Toolkit includes GPU-accelerated libraries, a compiler, development tools and the CUDA runtime.

## **Getting started**


In this tutorial, we will perform some operations with traditional computing and parallel computing. 

In [1]:
# import libraries
import numpy as np
from numpy import testing
from numba import cuda, types

## **Array summation: Traditional computing**

In this section, we will create two arrays using NumPy, we will sum them up and check the time.

In [2]:
# declare an array
n = 2048
x = np.arange(n).astype(np.int32) 
y = np.ones_like(x)   
z = np.zeros_like(x)            

In [3]:
# print out the result
print(x)
print(y)

[   0    1    2 ... 2045 2046 2047]
[1 1 1 ... 1 1 1]


In [4]:
# create a function that will sum up the arrays
def add_arr(x, y, z):
    for idx in range(0,len(x)):
      z[idx] = x[idx] + y[idx]
    return z

In [24]:
# print out time
%time
z = add_arr(x, y, z)
z

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


array([   1,    2,    3, ..., 2046, 2047, 2048], dtype=int32)

## **Array summation: Parallel computing**

Support for NumPy arrays is a key focus of Numba development and is currently undergoing extensive refactorization and improvement. Most capabilities of NumPy arrays are supported by Numba in object mode, and a few features are supported in nopython mode too (with much more to come).

A few noteworthy limitations of arrays at this time:

* Arrays can be passed in to a function in nopython mode, but not returned. Arrays can only be returned in object mode.
* New arrays can only be created in object mode.
* Currently there are no bounds checking for array indexing and slicing, although negative indices will wrap around correctly.
* A small number of NumPy array ufuncs are only supported in object mode, but the vast majority work in nopython mode.
* Array slicing only works in object mode.

In this section, we will create two arrays usign CUDA, we will sum them up and check the time.

### Compiling

CUDA kernels and device functions are compiled by decorating a Python function with the jit or autojit decorators. JIT compile a python function conforming to the CUDA-Python specification.

cuda.grid is a Numba-provided convenience function is equivalent to 
[cuda.threadIdx.x](https://numba.pydata.org/numba-doc/latest/cuda-reference/kernel.html#numba.cuda.threadIdx), [cuda.blockIdx.x](https://numba.pydata.org/numba-doc/latest/cuda-reference/kernel.html#numba.cuda.blockIdx), [cuda.blockDim.x](https://numba.pydata.org/numba-doc/latest/cuda-reference/kernel.html#numba.cuda.blockDim).

In [25]:
# define a kernel
@cuda.jit
def add_arr_parallel(x, y, out):
    idx = cuda.grid(1)      
    out[idx] = x[idx] + y[idx]

We have an input arrays of 2048 values, so we will use 4096 threads on the GPU. Our input and output arrays are one dimensional, so we will use a one-dimensional grid of threads (we will discuss grids in details in the next section). The call cuda.grid(1) returns the unique index for the current thread in the whole grid.  With 2048 threads, idx will range from 0 to 2047.

Then each thread is going to deal with a single element of the input array to produce a single element in the output array. This element is determined for each thread by the thread index.

### Memory transfer

Now that we have our kernel, we copy our input array to the GPU device, create an output array on the device with the same shape, and finally launch the kernel:

In [7]:
# allocate and transfer a numpy ndarray to copy the device.
d_x = cuda.to_device(x) 
d_y = cuda.to_device(y) 
# create output data on the device
d_z = cuda.device_array_like(d_x)

# lets use 16 blocks, each contains 128 threads 
threads_per_block = 128
blocks_per_grid = 16

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

### Execute the kernel

In [8]:
# launch the kernel
%%time
add_arr_parallel[blocks_per_grid, threads_per_block](d_x, d_y, d_z)

UsageError: Line magic function `%%time` not found.


In [9]:
# wait for all threads to complete
cuda.synchronize()
# copy the output array back to the host system and print it
d_copy = d_z.copy_to_host()
print(d_copy) 

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [10]:
testing.assert_array_equal(z, d_z)

NameError: name 'd_z' is not defined

## **Matrix multiplication: Traditional computing**

In [28]:
# declare matrices
M = 128
N = 32

a = np.arange(M*N).reshape(M,N).astype(np.int32)
b = np.arange(M*N).reshape(N,M).astype(np.inraditional cot32)
c = np.zeros((M, M)).astype(np.int32)

AttributeError: module 'numpy' has no attribute 'inraditional'

In [12]:
# create a function that will multiple two matrices
def mm(A,B,C):
  for i in range(len(A)):
      for j in range(len(B[0])):
          for k in range(len(B)):
              C[i][j] += A[i][k] * B[k][j]    
  return C

In [13]:
# print out time
%%time
output = mm(a,b,c)
output

UsageError: Line magic function `%%time` not found.


## **Matrix multiplication: Parallel computing (Striding)**

When the kernel is deployed, the GPU therefore needs to create as many threads as elements in the array, which potentially results in many blocks if the array is large.

On the contrary, a striding kernel deals with several elements of the input array, using a loop.

In this way, a given thread deals with several elements, and the number of threads is kept under control. Threads keep doing work in a coordinated way, and the GPU is not wasting time creating and scheduling threads.

In [14]:
# define a kernel
@cuda.jit
def mm_stride(A, B, C):

    grid_row, grid_column = cuda.grid(2)
    stride_row, stride_column = cuda.gridsize(2)
    
    for data_row in range(grid_column, A.shape[0], stride_column): 
        for data_column in range(grid_row, B.shape[1], stride_row): 
            sum = 0
            for i in range(A.shape[1]): 
                sum += A[data_row][i] * B[i][data_column]
            C[data_row][data_column] = sum

In [15]:
# allocate and transfer data back to the device
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

ts = (4, 3)
bs = (3, 7)

NameError: name 'a' is not defined

In [16]:
%%time
mm_stride[bs, ts](d_a, d_b, d_c)

NameError: name 'bs' is not defined

In [17]:
output_stride = d_c.copy_to_host()
output_stride

NameError: name 'd_c' is not defined

## **Matrix multiplication: Parallel computing (Shared memory)**

A limited amount of shared memory can be allocated on the device to speed up access to data, when necessary. That memory will be shared (i.e. both readable and writable) amongst all threads belonging to a given block and has faster access times than regular device memory. It also allows threads to cooperate on a given solution. You can think of it as a manually-managed data cache.

The memory is allocated once for the duration of the kernel, unlike traditional dynamic memory management.

In [29]:
# NxN threads per block, in 2 dimensions
block_size = (N,N)
# MxM/NxN blocks per grid, in 2 dimensions
grid_size = (int(M/N),int(M/N))

In [30]:
@cuda.jit

def mm_shared(a, b, c):
    x, y = cuda.grid(2)
    TPB = N # threads per block
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    a_cache = cuda.shared.array(block_size, types.int32)
    b_cache = cuda.shared.array(block_size, types.int32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        a_cache[tx, ty] = 0
        b_cache[tx, ty] = 0
        if x < a.shape[0] and (ty+i*TPB) < a.shape[1]:
            a_cache[tx, ty] = a[x, ty + i * TPB]
        if y < b.shape[1] and (tx+i*TPB) < b.shape[0]:
            b_cache[tx, ty] = b[tx + i * TPB, y]

        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += a_cache[tx, j] * b_cache[j, ty]

        cuda.syncthreads()
    if x < c.shape[0] and y < c.shape[1]:
        c[x, y] = tmp

In [31]:
%%time
mm_shared[grid_size, block_size](d_a, d_b, d_c)

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [32]:
output_shared = d_c.copy_to_host()
output_shared

NameError: name 'd_c' is not defined

In [22]:
testing.assert_array_equal(output, output_stride)

NameError: name 'output' is not defined

In [23]:
testing.assert_array_equal(output, output_shared)

NameError: name 'output' is not defined